In [1]:
# %load_ext tensorboard
# Clear any logs from previous runs
# !rm -rf ./logs/

In [2]:
import numpy as np
import pandas as pd
import time, os

import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.layers import Input, Dense
from tensorflow.keras.utils import to_categorical
from tensorboard.plugins.hparams import api as hp

from tensorflow.keras.losses import CategoricalCrossentropy
from tensorflow.keras.models import Model

from sklearn.preprocessing import StandardScaler, MinMaxScaler
from sklearn.model_selection import train_test_split as split

import random
from math import floor
from tensorflow.keras import models
from tensorflow.keras import layers
from tensorflow.keras import optimizers
from scipy.integrate import odeint
import scipy
import pandas as pd
np.random.seed(0)

2022-12-16 16:43:49.664360: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2022-12-16 16:43:49.799241: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2022-12-16 16:43:49.799260: I tensorflow/compiler/xla/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.
2022-12-16 16:43:50.743668: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer.so.7'; dlerror: libnvinfer.so.7: cannot open shared object file: No such file or directory
2022-

In [3]:
url = 'https://raw.githubusercontent.com/igomezv/nnogada/main/data/jla.csv'
df=pd.read_csv(url)
N = len(df.values)
randomize = np.random.permutation(N)
data = df.values[randomize]
N = len(df.values)
z = data[:,0] 
y = data[:,1:3] ### coge el resto de variables a predecir 
y[:,1] = y[:,1]**2+data[:,2]
np.shape(y)

(740, 2)

In [4]:
scalerz = StandardScaler()
scalerz.fit(z.reshape(-1,1))
z = scalerz.transform(z.reshape(-1,1))

In [5]:
### Modificar para incluir phanteon como test
split = 0.8
ntrain = int(split * len(z))
indx = [ntrain]
X_train, X_test = np.split(z, indx)
Y_train, Y_test = np.split(y, indx)
np.shape(X_train), np.shape(X_test), Y_train.shape, Y_test.shape

((592, 1), (148, 1), (592, 2), (148, 2))

In [6]:
# loss -> val_loss
callbacks = [tf.keras.callbacks.EarlyStopping(monitor='val_loss', mode='min',
                                   min_delta=0.0,
                                   patience=200,
                                   restore_best_weights=True, verbose=True)
                                   ]

n_cols = 1

In [7]:
HP_BATCHSIZE = hp.HParam('batch_size', hp.Discrete([2, 4, 8, 16]))
HP_LAYERS =    hp.HParam('layers', hp.Discrete([1, 2, 3, 4]))
HP_NUM_UNITS = hp.HParam('num_units', hp.Discrete([50, 100, 150, 200]))
HP_LEARNING  = hp.HParam('learning_rate', hp.Discrete([1e-4,1e-3]))
# HP_NUM_UNITS3 = hp.HParam('num_units3', hp.Discrete([50, 100, 150, 200]))
# HP_NUM_UNITS4 = hp.HParam('num_units4', hp.Discrete([2, 5, 10]))
# HP_DROPOUT = hp.HParam('dropout', hp.RealInterval(0.0, 0.2))
# HP_OPTIMIZER = hp.HParam('optimizer', hp.Discrete(['adam', 'sgd']))
# HP_OPTIMIZER = hp.HParam('optimizer', hp.Discrete(['adam', 'Adadelta']))
# HP_OPTIMIZER = hp.HParam('optimizer', hp.Discrete(['adam']))

In [8]:
epochs=200

In [9]:
# sess = tf.Session()
# writer = tf.summary.FileWriter("/tmp/tfvgg", sess.graph)
# init = tf.initialize_all_variables()
# sess.run(init)
# with tf.summary.create_file_writer('logs/hparam_tuning').as_default():
#     hp.hparams_config(
#         hparams=[HP_NUM_UNITS1, HP_NUM_UNITS2, HP_NUM_UNITS3, HP_NUM_UNITS4,
#                  HP_OPTIMIZER, HP_BATCHSIZE],
#         metrics=[hp.Metric('loss', display_name="Loss")])

In [10]:
# METRIC_ACCURACY = 'accuracy'
with tf.summary.create_file_writer('logs/hparam_tuning3').as_default():
# with tf.summary.FileWriter('logs/hparam_tuning', sess.graph):
#     init = tf.initialize_all_variables()
#     sess.run(init)
    hp.hparams_config(
        hparams=[HP_LAYERS,
                 HP_NUM_UNITS,
                 HP_LEARNING, 
                 HP_BATCHSIZE],
        metrics=[hp.Metric('loss', display_name="Loss")])

2022-12-16 16:43:52.104427: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:981] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-12-16 16:43:52.104635: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2022-12-16 16:43:52.104702: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcublas.so.11'; dlerror: libcublas.so.11: cannot open shared object file: No such file or directory
2022-12-16 16:43:52.104766: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcublasLt.so.11'; dlerror: libcublasLt.so.11: cannot open shared object file: No such file or directory
2022-12-16 16:43:52.104833: W tensorflow/c

In [11]:
def train_test_model(hparams):    
    
    # Train LSTM model and predict on validation set
    model = tf.keras.Sequential()
    model.add(Dense(int(X_train.shape[1])))
#     model.add(Dense(hparams[HP_NUM_UNITS], input_shape=(int(X_train.shape[1]),)))
    
    for i in range(hparams[HP_LAYERS]):        
        model.add(Dense(hparams[HP_NUM_UNITS], activation='relu'))
    model.add(Dense(2, activation='linear'))
     
    optimizer = tf.keras.optimizers.Adam(learning_rate=hparams[HP_LEARNING], beta_1=0.9, beta_2=0.999, epsilon=1e-3)
    model.compile(
            optimizer=optimizer,
            loss='mse', 
            metrics=['mean_squared_error'])
    
    # Run with 1 epoch to speed things up for demo purposes

    model.fit(X_train, Y_train, epochs=epochs, validation_data=(X_test, Y_test),
              callbacks=callbacks, batch_size=hparams[HP_BATCHSIZE], shuffle=False, verbose=0)

    _, loss = model.evaluate(X_test, Y_test)
    
    return loss

In [12]:
def run(run_dir, hparams):
    with tf.summary.create_file_writer(run_dir).as_default():
        hp.hparams(hparams)  # record the values used in this trial
        loss = train_test_model(hparams)
        tf.summary.scalar("loss", loss, step=1)
        return loss

In [13]:
session_num = 0
datos = []

for deep_layers in HP_LAYERS.domain.values:
    for num_units in HP_NUM_UNITS.domain.values:
        for learning_rate in HP_LEARNING.domain.values:
            for batch_size in HP_BATCHSIZE.domain.values:
                t = time.time()
                hparams = {

                    HP_LAYERS: deep_layers,
                    HP_NUM_UNITS: num_units,
                    HP_LEARNING: learning_rate,
                    HP_BATCHSIZE: batch_size,
                }
                run_name = "run-%d" % session_num
                print('\n--- Starting trial: %s' % run_name)
                print({h.name: hparams[h] for h in hparams})
                score = run('logs/hparam_tuning3/' + run_name, hparams)
                t = time.time()-t
                session_num += 1
                print("Loss:", score, "Tiempo transcurrido:", t)
            
            datos.append([deep_layers, num_units, learning_rate, batch_size, score, t])

print(session_num)


--- Starting trial: run-0
{'layers': 1, 'num_units': 50, 'learning_rate': 0.0001, 'batch_size': 2}
5/5 [==============================] - 0s 2ms/step - loss: 0.0621 - mean_squared_error: 0.0621
Loss: 0.06205371767282486 Tiempo transcurrido: 98.10645055770874

--- Starting trial: run-1
{'layers': 1, 'num_units': 50, 'learning_rate': 0.0001, 'batch_size': 4}
5/5 [==============================] - 0s 2ms/step - loss: 0.1203 - mean_squared_error: 0.1203
Loss: 0.1203412264585495 Tiempo transcurrido: 53.0462965965271

--- Starting trial: run-2
{'layers': 1, 'num_units': 50, 'learning_rate': 0.0001, 'batch_size': 8}
5/5 [==============================] - 0s 2ms/step - loss: 0.2490 - mean_squared_error: 0.2490
Loss: 0.24895435571670532 Tiempo transcurrido: 30.353832244873047

--- Starting trial: run-3
{'layers': 1, 'num_units': 50, 'learning_rate': 0.0001, 'batch_size': 16}
5/5 [==============================] - 0s 2ms/step - loss: 6.5854 - mean_squared_error: 6.5854
Loss: 6.585424900054932 T

5/5 [==============================] - 0s 2ms/step - loss: 0.0395 - mean_squared_error: 0.0395
Loss: 0.03949914500117302 Tiempo transcurrido: 20.77964425086975

--- Starting trial: run-32
{'layers': 2, 'num_units': 50, 'learning_rate': 0.0001, 'batch_size': 2}
5/5 [==============================] - 0s 2ms/step - loss: 0.0346 - mean_squared_error: 0.0346
Loss: 0.03456685319542885 Tiempo transcurrido: 143.23218274116516

--- Starting trial: run-33
{'layers': 2, 'num_units': 50, 'learning_rate': 0.0001, 'batch_size': 4}
5/5 [==============================] - 0s 2ms/step - loss: 0.0396 - mean_squared_error: 0.0396
Loss: 0.039649561047554016 Tiempo transcurrido: 57.16809010505676

--- Starting trial: run-34
{'layers': 2, 'num_units': 50, 'learning_rate': 0.0001, 'batch_size': 8}
5/5 [==============================] - 0s 3ms/step - loss: 0.0674 - mean_squared_error: 0.0674
Loss: 0.06735040992498398 Tiempo transcurrido: 36.64599895477295

--- Starting trial: run-35
{'layers': 2, 'num_units': 

5/5 [==============================] - 0s 1ms/step - loss: 0.0322 - mean_squared_error: 0.0322
Loss: 0.03223836421966553 Tiempo transcurrido: 23.921727895736694

--- Starting trial: run-63
{'layers': 2, 'num_units': 200, 'learning_rate': 0.001, 'batch_size': 16}
5/5 [==============================] - 0s 1ms/step - loss: 0.0536 - mean_squared_error: 0.0536
Loss: 0.05364613980054855 Tiempo transcurrido: 15.992690563201904

--- Starting trial: run-64
{'layers': 3, 'num_units': 50, 'learning_rate': 0.0001, 'batch_size': 2}
5/5 [==============================] - 0s 1ms/step - loss: 0.0309 - mean_squared_error: 0.0309
Loss: 0.030946731567382812 Tiempo transcurrido: 61.80705118179321

--- Starting trial: run-65
{'layers': 3, 'num_units': 50, 'learning_rate': 0.0001, 'batch_size': 4}
5/5 [==============================] - 0s 1ms/step - loss: 0.0365 - mean_squared_error: 0.0365
Loss: 0.03648320212960243 Tiempo transcurrido: 32.22715711593628

--- Starting trial: run-66
{'layers': 3, 'num_units'

5/5 [==============================] - 0s 1ms/step - loss: 0.0319 - mean_squared_error: 0.0319
Loss: 0.03194231912493706 Tiempo transcurrido: 39.13094139099121

--- Starting trial: run-94
{'layers': 3, 'num_units': 200, 'learning_rate': 0.001, 'batch_size': 8}
5/5 [==============================] - 0s 1ms/step - loss: 0.0448 - mean_squared_error: 0.0448
Loss: 0.044801026582717896 Tiempo transcurrido: 23.628043174743652

--- Starting trial: run-95
{'layers': 3, 'num_units': 200, 'learning_rate': 0.001, 'batch_size': 16}
5/5 [==============================] - 0s 1ms/step - loss: 0.0325 - mean_squared_error: 0.0325
Loss: 0.032527852803468704 Tiempo transcurrido: 15.28300929069519

--- Starting trial: run-96
{'layers': 4, 'num_units': 50, 'learning_rate': 0.0001, 'batch_size': 2}
5/5 [==============================] - 0s 1ms/step - loss: 0.0315 - mean_squared_error: 0.0315
Loss: 0.03145481273531914 Tiempo transcurrido: 63.310216188430786

--- Starting trial: run-97
{'layers': 4, 'num_units

5/5 [==============================] - 0s 1ms/step - loss: 0.1034 - mean_squared_error: 0.1034
Loss: 0.1034083217382431 Tiempo transcurrido: 82.1416757106781

--- Starting trial: run-125
{'layers': 4, 'num_units': 200, 'learning_rate': 0.001, 'batch_size': 4}
5/5 [==============================] - 0s 2ms/step - loss: 0.0380 - mean_squared_error: 0.0380
Loss: 0.03803648427128792 Tiempo transcurrido: 44.59988570213318

--- Starting trial: run-126
{'layers': 4, 'num_units': 200, 'learning_rate': 0.001, 'batch_size': 8}
5/5 [==============================] - 0s 1ms/step - loss: 0.0344 - mean_squared_error: 0.0344
Loss: 0.03444308415055275 Tiempo transcurrido: 27.330164194107056

--- Starting trial: run-127
{'layers': 4, 'num_units': 200, 'learning_rate': 0.001, 'batch_size': 16}
5/5 [==============================] - 0s 1ms/step - loss: 0.0445 - mean_squared_error: 0.0445
Loss: 0.044546470046043396 Tiempo transcurrido: 16.966399669647217
128


In [14]:
filename = "historial_jla_tunning.txt"
df = pd.DataFrame(datos, columns = ["Deep size", "Num units", "Learning rate", "Batch size", "MSE", "Tiempo de ejecución"])

df.sort_values(by=["MSE", "Tiempo de ejecución"], ascending=[True, True], ignore_index=True, inplace=True)

df.to_csv(filename, header=True, index=False, sep='\t', mode='w') # a=append, w=overwrite

In [15]:
df.head(2)

,Deep size,Num units,Learning rate,Batch size,MSE,Tiempo de ejecución
0,3,200,0.0010,16,0.032528,15.283009
1,4,200,0.0001,16,0.034978,16.996224


In [16]:
total_time = float(np.sum(df[["Tiempo de ejecución"]])/60)

print("Elapsed time {:.3f} minutes".format(total_time))

Elapsed time 9.060 minutes


In [17]:
df["Tiempo de ejecución"]

0     15.283009
1     16.996224
2     15.755800
3     20.696011
4     13.272431
5     14.016798
6     15.387176
7     18.736623
8     14.465001
9     20.815081
10    14.132247
11    20.779644
12    20.273613
13    16.088559
14    13.779431
15    16.966400
16    16.754388
17    13.201734
18    17.425991
19    15.992691
20    12.418356
21    17.909512
22    15.171564
23    12.726302
24    13.515701
25    21.273691
26    12.948186
27    24.705475
28    21.890503
29    22.035712
30    20.444350
31    17.770208
Name: Tiempo de ejecución, dtype: float64